## Initialization

In [256]:
import pandas as pd
import requests
import urllib
import re
import os

from bs4 import BeautifulSoup

## Read dataset

In [257]:
if os.path.exists('dataset/kaggle_beer.csv'):
    data = pd.read_csv('dataset/kaggle_beer.csv', error_bad_lines=False)

# Subset only USA brewed beers    
data = data[data.country == 'United States']

In [258]:
# Print the size of the dataset
print ("Number of rows:", data.shape[0])
print ("Number of columns: ", data.shape[1])

Number of rows: 4588
Number of columns:  22


## Webscrapped www.beeradvocate.com to comple kaggle beer dataset

In [259]:
#webscrapping beeradvocate
all_beers = []
counter = 0

for index, row in data.iterrows():

    avg_score = num_ratings = style = availability = -1
    
    counter = counter + 1
    if counter%1000 == 0:
        print(counter)
    
    beer_name_og = row['beer_name']
    style_og = row['style']
    category_og = row['category']
    brewery_og = row['brewery']
    city_og = row['city']
    state_og = row['state']
    
    base_url = "https://www.beeradvocate.com"
    search_url = base_url + "/search/?qt=beer&q="
    
    if len(brewery_og.split()) > 1:
        search_beer_brewery = beer_name_og + ' ' + brewery_og.split()[0] + ' ' + brewery_og.split()[1]
    else:
        search_beer_brewery = beer_name_og + ' ' + brewery_og
    
    query_url = search_url + urllib.parse.quote(search_beer_brewery)
    page = requests.get(query_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Search result page
    query_title = soup.title
    if (query_title == "Search | BeerAdvocate"):
        first_beer_result = soup.find(id="ba-content").find_all('a')[0]
        first_beer_result_name = first_beer_result.string
        first_beer_result_relative_link = first_beer_result.get('href')

        # Beer profile page
        beer_url = base_url + first_beer_result_relative_link
        page = requests.get(beer_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        
    # Get beer profile information
    try:
        ba_info = soup.find(id="info_box").text
    except:
        continue
        
    avg_score = soup.find(class_="ba-ravg").text
    num_ratings = soup.find(class_="ba-ratings").text
    style = re.search('Style: (.*)',ba_info)[1]
    avail = re.search('Availability: (.*)',ba_info)[1]
    try:
        abv = re.search(': (.*)%',soup.find(id="info_box").text)[1]
    except:
        abv = -1
    try: 
        ibu = re.search('(.*) IBU',soup.find(id="info_box").text)[1]
    except: 
        ibu = -1

    new_entry = {
        'beer_name_og' : beer_name_og,
        'avg_score' : avg_score,
        'num_of_ratings' : num_ratings,
        'style' : style,
        'avail' : avail,
        'abv' : abv,
        'ibu' : ibu,
        'style_og' : style_og,
        'category_og' : category_og,
        'brewery_og' : brewery_og,
        'city_og' : city_og,
        'state_og' : state_og     
    }
    
    all_beers.append(new_entry)

1000
2000
3000
4000


## Convert to pandas dataframe

In [260]:
final = pd.DataFrame(all_beers)

In [280]:
final.head(100)

,abv,avail,avg_score,beer_name_og,brewery_og,category_og,city_og,ibu,num_of_ratings,state_og,style,style_og
0,5.00,Year-round,4.17,Scottish Ale,Carlyle Brewing,British Ale,Rockford,-1,7,Illinois,Scottish Ale,Scotch Ale
1,5.90,Year-round,3.48,Bee Sting Honey Ale,Great Divide Brewing,North American Ale,Denver,-1,41,Colorado,American Pale Ale (APA),American-Style Pale Ale
2,5.00,Year-round,3.39,Native Ale,New Glarus Brewing Company,North American Ale,New Glarus,-1,56,Wisconsin,English Brown Ale,American-Style Amber/Red Ale
3,-1,Rotating,3.33,New Peculier,Great Dane Pub and Brewing #2,British Ale,Fitchburg,-1,8,Wisconsin,American Brown Ale,Old Ale
4,5.50,Year-round,3.72,Old Glory American Pale Ale,Great Dane Pub and Brewing #2,North American Ale,Fitchburg,-1,79,Wisconsin,American Pale Ale (APA),American-Style Pale Ale
5,-1,Year-round,3.16,Canadian Light,Courthouse Pub,North American Lager,Manitowoc,-1,1,Wisconsin,American Light Lager,American-Style Lager
6,-1,Rotating,3.04,Honey Wheat,Great Waters Brewing Company,North American Lager,Saint Paul,-1,3,Minnesota,American Pale Wheat Ale,American-Style Lager
7,-1,Year-round,3.49,Stillwater Stout,Rock Bottom Restaurant & Brewery - Minneapolis,North American Ale,Minneapolis,-1,22,Minnesota,American Stout,American-Style Stout
8,-1,Year-round,3.97,Itasca Extra Pale Ale,Rock Bottom Restaurant & Brewery - Minneapolis,British Ale,Minneapolis,-1,28,Minnesota,American Pale Ale (APA),English-Style Pale Mild Ale
9,-1,Year-round,3.56,Light,Wild River Brewing and Pizza - Cave Junction,NaN,Cave Junction,-1,9,Oregon,German Kölsch,NaN


In [264]:
# Print the size of the dataset
print ("Number of rows:", final.shape[0])
print ("Number of columns: ", final.shape[1])

Number of rows: 1723
Number of columns:  12


## Clean

In [303]:
import nltk
import string
from string import digits

In [304]:
remove_punctuation = str.maketrans('', '', string.punctuation)
remove_digits = str.maketrans('', '', digits)

In [305]:
def clean_column(df,colname):
    df[colname] = df[colname].str.translate(remove_punctuation)

In [284]:
ibu_filter = final.ibu.str.contains('[a-zA-Z]', regex=True)
ibu_filter = ibu_filter.fillna(False)

In [ ]:
clean_final = final.loc[~ibu_filter]

In [306]:
clean_column(clean_final,'num_of_ratings')

C:\Users\pmann\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [307]:
clean_final['num_of_ratings'] = pd.to_numeric(clean_final['num_of_ratings'])

C:\Users\pmann\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [309]:
# Print the size of the dataset
print ("Number of rows:", clean_final.shape[0])
print ("Number of columns: ", clean_final.shape[1])

Number of rows: 1705
Number of columns:  12


## Export to CSV

In [310]:
export_csv = clean_final.to_csv (r'dataset\beer_advocate_webscrapped.csv', index = None, header=True)